In [1]:
import numpy as np
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import tensorflow as tf

In [2]:
len_vcb = 366549
seed = 23
max_len = 150

In [3]:
with open('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/utils/tokenizer_100k.json') as f: 
    data = json.load(f) 
    tok = tokenizer_from_json(data)

In [4]:
### definindo metricas customizadas

def recall(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # transformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_recall = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_pred[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_possibles = y_true[K.equal(y_true,K.constant(i))]/i
        possible_positives = K.sum(equals_possibles)

        recall = true_positives / (possible_positives + K.epsilon())
        macro_recall = macro_recall + recall

    macro_recall = macro_recall/(n_tags-1)  

    return macro_recall

def precision(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # trannsformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_precision = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_true[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_pred_possibles = y_pred[K.equal(y_pred,K.constant(i))]/i
        predicted_positives = K.sum(equals_pred_possibles)

        precision = true_positives / (predicted_positives + K.epsilon())
        macro_precision = macro_precision + precision

    macro_precision = macro_precision/(n_tags-1)

    return macro_precision

def macro_f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [5]:
custom_metrics = {"recall": recall,"precision": precision, "macro_f1": macro_f1}
model = tf.keras.models.load_model(
    '/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/SP_TOTAL_1EPOCH.h5',
    custom_objects=custom_metrics)

In [6]:
alagoas_df = pd.read_json('/home/rafael/Documents/Faculdade/ICs/IC Direito/data/samples/sample_alagoas_edital.json', lines=True)    

In [7]:
alagoas_sample = alagoas_df.sample(1, random_state=seed)['conteudo'].to_list()
alagoas_sample

['ADV: JAIME MARÇAL DANTAS FILHO (OAB 33947/PE), RODRIGO DE LIMA COSTA (OAB 10167/AL) - Processo 0704231- 05.2017.8.02.0001 - Procedimento Ordinário - Seguro - AUTOR: José da Silva - Indeﬁro os benefícios da Assistência Judiciária Gratuita, com fulcro no artigo 98 e seguintes do novo CPC e Lei n.º 1.060/50. Por outro lado, deﬁro o pagamento das custas ao ﬁnal do processo. Por ﬁm, visando cumprir o artigo 334 e seguintes do novo CPC, remetam-se os autos ao CJUS, para ﬁns de designação de audiência preliminar de conciliação.Maceió(AL), 15 de março de 2017.Maria Valéria Lins Calheiros Juíza de Direito']

In [8]:
alagoas_sample_tokenized = tok.texts_to_sequences(alagoas_sample)
pd.DataFrame(alagoas_sample_tokenized[0]).T

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,18,7765,11534,4907,559,10,1,4819,745,22,...,975,2,220,254,13174,9022,107834,2263,2,166


In [9]:
alagoas_sample_tokenized_padded = pad_sequences(maxlen=max_len,
                                    sequences=alagoas_sample_tokenized,
                                    padding="post",
                                    truncating="post")
pd.DataFrame(alagoas_sample_tokenized_padded)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,18,7765,11534,4907,559,10,1,4819,745,22,...,0,0,0,0,0,0,0,0,0,0


In [10]:
predicted_alagoas_sample_tokenized_padded = model(alagoas_sample_tokenized_padded)

In [11]:
for i,p in enumerate(predicted_alagoas_sample_tokenized_padded[0]):
    if (i < len(alagoas_sample_tokenized[0])): 
        print(np.argmax(p),tok.index_word[alagoas_sample_tokenized[0][i]])

1 ADV
1 JAIME
1 MARÇAL
1 DANTAS
1 FILHO
1 OAB
1 UNK
1 PE
1 RODRIGO
1 DE
1 LIMA
1 COSTA
1 OAB
1 UNK
1 AL
1 Processo
1 UNK
1 05
1 2017
1 8
1 02
1 0001
14 Procedimento
2 Ordinário
1 Seguro
1 AUTOR
1 José
1 da
1 Silva
1 UNK
1 os
1 benefícios
1 da
1 Assistência
1 Judiciária
1 Gratuita
1 com
1 fulcro
1 no
1 artigo
1 98
1 e
1 seguintes
1 do
1 novo
1 CPC
1 e
1 Lei
1 n
1 º
1 1
1 060
1 50
1 Por
1 outro
1 lado
1 UNK
1 o
1 pagamento
1 das
1 custas
1 ao
1 UNK
1 do
1 processo
1 Por
1 UNK
1 visando
1 cumprir
1 o
1 artigo
1 334
1 e
1 seguintes
1 do
1 novo
1 CPC
1 remetam
1 se
1 os
1 autos
1 ao
1 UNK
1 para
1 UNK
1 de
1 designação
1 de
1 audiência
1 preliminar
1 de
1 conciliação
1 Maceió
1 AL
1 15
1 de
1 março
1 de
1 2017
1 Maria
1 Valéria
1 Lins
1 Calheiros
1 Juíza
1 de
1 Direito
